The purpose of this notebook is to calculate the power produced by a decay chain after the production of a given nuclide.

In [58]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Downloads
#plotlyjs()
data_dir = ("C:\\Cross-Section-Data\\")
cross_section_dir = data_dir
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"
endf8_decay_dir = cross_section_dir * "ENDF_Libraries-2\\ENDF-B-VIII.0\\decay\\"
jeff33_decay_dir = cross_section_dir * "ENDF_Libraries-2\\JEFF-3.3\\decay\\"
jendl5_decay_dir = cross_section_dir * "jendl5-dec_upd5\\"

"C:\\Cross-Section-Data\\jendl5-dec_upd5\\"

Now I will find the average decay energies for each nuclide. For the datasets I am using, the average decay energies are all given in keV. 

In [59]:
function get_mass_name(nuclide)
    index = 1
    while (tryparse(Int64, string(nuclide[index])) != nothing && index <= length(nuclide))
        index += 1
    end
    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
element_symbols = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", 
"Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", 
"Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", 
"Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", 
"Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", 
"Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", 
"Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", 
"Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", 
"Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", 
"Hs", "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"];

Rename files in directory to be easier to parse by removing zero-padding.

In [60]:
function unzero_pad(num_str) 
    if (num_str == '0')
        return num_str
    end
    i = 1
    while i < length(num_str) && num_str[i] == '0'
        i += 1
    end
    return num_str[i : end]
end
function rename_files_in_dir(dir, cutoff_index)
    for f in readdir(dir)
        if tryparse(Int64, string(f[1])) == nothing
            mv(dir * f, dir * f[cutoff_index:end])
        end
        if f != join(unzero_pad.(split(f, "-")), "-")
            mv(dir * f, dir * join(unzero_pad.(split(f, "-")), "-"))
        end
    end
end
rename_files_in_dir(endf8_decay_dir, 12)
rename_files_in_dir(jeff33_decay_dir, 12)
rename_files_in_dir(jendl5_decay_dir, 4)

In [61]:
function get_decay_energy_from_file(file_as_arr, search_string)
    line = [l for l in file_as_arr if length(l) > length(search_string)
             && occursin(search_string, l)]
    if (length(line) > 0)
        line = String(line[1])
        line = join(split(line, search_string, keepempty = false))
        return tryparse(Float64, join(split(line, " ", keepempty = false)[1]))
    else 
        return "Energy not found"
    end
end

get_decay_energy_from_file (generic function with 1 method)

This dictionary stores what strings to look to for in the datasets to find the average energy for each decay type, and for each directory. 

In [62]:
dir_search_string_dict = Dict([
    jeff33_decay_dir => Dict(["alpha" => "ALPHA ENERGY                   =",
    "beta-minus" => "MEAN BETA- ENERGY              =","gamma" => "MEAN GAMMA ENERGY              =", 
    "x-ray" => "MEAN X-RAY ENERGY              =", "auger" => "MEAN AUGER ELECTRON ENERGY     ="
    ]),

    endf8_decay_dir => Dict(["alpha" => "Mean Alpha Energy:", 
    "beta-minus" => "Mean B- Energy:", "beta-plus" => "Mean B+ Energy:",
    "gamma" => "Mean Gamma Energy:", "auger" => "Mean CE+Auger Energy:",
    "neutron" => "Mean Neutron Energy:", "proton" => "Mean Neutron Energy:"]),

    jendl5_decay_dir => Dict(["alpha" => "Mean Alpha Energy",
    "beta-minus" => "Mean B- Energy", "gamma" => "Mean Gamma Energy",
    "neutron" => "Mean Neutron Energy", "proton" => "Mean Proton Energy",
    "auger" => "Mean CE+Auger Energy"])  
])

Dict{String, Dict{String, String}} with 3 entries:
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy", "proton"=>…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"Mean CE+Auger Energy:", "proton"=…
  "C:\\Cross-Section-Data\… => Dict("auger"=>"MEAN AUGER ELECTRON ENERGY     ="…

In [63]:
function find_avg_decay_energy(nuclide, decay_type, dir)
    a, el = get_mass_name(nuclide)
    el = string(el)
    z = [i for i in 1:length(element_symbols) if element_symbols[i] == el][1]
    file_name = string(z) * "-" * el * "-" * string(a) * ".dat"
    file_path = dir * file_name
    file_as_array = split(open(f->read(f, String), file_path), "\n")
    search_string = dir_search_string_dict[dir][decay_type]
    return get_decay_energy_from_file(file_as_array, search_string)
end
dir_list = keys(dir_search_string_dict)
function find_avg_decay_energy(nuclide, decay_type)
    for dir in dir_list
        energy = find_avg_decay_energy(nuclide, decay_type, dir)
        if energy != "Energy not found"
            return energy
        end
    end
    return "Energy not found"
end

find_avg_decay_energy (generic function with 2 methods)

Now for some examples. Again, all average decay energies are given in keV. 

In [64]:
out = find_avg_decay_energy("238U", "alpha", jeff33_decay_dir)

4187.0737

In [65]:
out = find_avg_decay_energy("3H", "beta-minus", jendl5_decay_dir)

5.69

In [66]:
avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jendl5_decay_dir)

"Energy not found"

In [67]:
avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jendl5_decay_dir)
if avg_decay_energy == "Energy not found"
    avg_decay_energy = find_avg_decay_energy("8Be", "alpha", jeff33_decay_dir)
end

45.95

In [68]:
find_avg_decay_energy("238U", "alpha", endf8_decay_dir)

4187.0

In [69]:
live_chart_file_name = "all_livechart_decays_and_emissions.csv"
all_live_chart_data = CSV.read(parent_dir * "ImportedData\\" * live_chart_file_name,
                        DataFrame, stringtype = String)[:,2:end]

,z,n,symbol,radius,unc_r,abundance,abundance_unc,energy_shift
,Int64,Int64,String,String,Float64?,String,String?,String?
1,0,1,Nn,-0.1149,0.0027,,missing,
2,0,4,N,,missing,,missing,
3,0,6,N,,missing,,missing,
4,1,0,H,0.8783,0.0086,99.9855,78,
5,1,1,H,2.1421,0.0088,0.0145,78,
6,1,2,H,1.7591,0.0363,,missing,
7,1,3,H,,missing,,missing,
8,1,4,H,,missing,,missing,
9,1,5,H,,missing,,missing,


In [70]:
all_live_chart_data[!, "nuclide"] = [string(all_live_chart_data[row, "z"] + 
                        all_live_chart_data[row, "n"]) * all_live_chart_data[row, "symbol"]
                        for row in 1:size(all_live_chart_data)[1]]
livechart_nuclides = unique(all_live_chart_data[!, "nuclide"])

3367-element Vector{String}:
 "1Nn"
 "4N"
 "6N"
 "1H"
 "2H"
 "3H"
 "4H"
 "5H"
 "6H"
 "7H"
 "3He"
 "4He"
 "5He"
 ⋮
 "287Mc"
 "288Mc"
 "289Mc"
 "290Mc"
 "290Lv"
 "291Lv"
 "292Lv"
 "293Lv"
 "294Lv"
 "293Ts"
 "294Ts"
 "294Og"

In [71]:
top_decay_probs = [tryparse(BigFloat, p) for p in all_live_chart_data[!,"decay_1_%"] 
                    if tryparse(Float64, p) != nothing]
decay_prob_df = DataFrame(Prob = top_decay_probs)

,Prob
,BigFloat
1,100.0
2,100.0
3,100.0
4,100.0
5,100.0
6,100.0
7,100.0
8,100.0
9,100.0


In [72]:
plot(decay_prob_df, x=:Prob, kind = "histogram", nbinsx=100, 
Layout(yaxis_type = "log"))

data: [
  "histogram with fields nbinsx, type, x, xaxis, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

In [73]:
names(all_live_chart_data)[1:20]

20-element Vector{String}:
 "z"
 "n"
 "symbol"
 "radius"
 "unc_r"
 "abundance"
 "abundance_unc"
 "energy_shift"
 "energy"
 "unc_e"
 "ripl_shift"
 "jp"
 "half_life"
 "operator_hl"
 "unc_hl"
 "unit_hl"
 "half_life_sec"
 "unc_hls"
 "decay_1"
 "decay_1_%"

In [74]:
all_live_chart_data[!, "Decay or Emission type"]

20202-element PooledArrays.PooledVector{String, UInt32, Vector{UInt32}}:
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 ⋮
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"

In [75]:
all_live_chart_data[!, "half_life_sec"]

20202-element Vector{String}:
 "613.9"
 "1.7547604425822527e-22"
 " "
 " "
 " "
 "388781328.00697345"
 " "
 "8.608258774931805e-23"
 "2.943469129492811e-22"
 "5.069307945237619e-21"
 " "
 " "
 "7.040705479496693e-22"
 ⋮
 " "
 "0.17"
 "0.22"
 "0.65"
 "0.0083"
 " "
 "0.0128"
 "0.053"
 " "
 "0.014"
 "0.051"
 "0.00058"

In [76]:
decay_or_emission_type_dict = Dict([
    "B-" =>  "beta minus decay", "A" =>  "alpha decay", "B+" =>  "beta plus decay",
    "EC" => "auger and conversion electron"
])
function get_half_life(nuclide, decay_type)
    if decay_type in keys(decay_or_emission_type_dict)
        decay_type_long = decay_or_emission_type_dict[decay_type]
        row = [row for row in 1:size(all_live_chart_data)[1]
                if ((all_live_chart_data[row, "nuclide"] ==  nuclide) && 
                (decay_type_long == all_live_chart_data[row, "Decay or Emission type"])
                )][1]
        return all_live_chart_data[row, "half_life_sec"]
    end
    if decay_type in values(decay_or_emission_type_dict)
        row = [row for row in 1:size(all_live_chart_data)[1]
                if (all_live_chart_data[row, "nuclide"] ==  nuclide && 
                decay_type == all_live_chart_data[row, "Decay or Emission type"])][1]
        return all_live_chart_data[row, "half_life_sec"]
    end
    return decay_type * " not accounted for "
end

get_half_life (generic function with 1 method)

In [77]:
all_live_chart_data[!, "nuclide"] 

20202-element Vector{String}:
 "1Nn"
 "4N"
 "6N"
 "1H"
 "2H"
 "3H"
 "4H"
 "5H"
 "6H"
 "7H"
 "3He"
 "4He"
 "5He"
 ⋮
 "287Mc"
 "288Mc"
 "289Mc"
 "290Mc"
 "290Lv"
 "291Lv"
 "292Lv"
 "293Lv"
 "294Lv"
 "293Ts"
 "294Ts"
 "294Og"

In [78]:
all_live_chart_data[!, "Decay or Emission type"]

20202-element PooledArrays.PooledVector{String, UInt32, Vector{UInt32}}:
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 "beta minus decay"
 ⋮
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"
 "x ray emission"

In [79]:
function get_daughter_nuclide(parent, decay_type)
    if decay_type == "gamma"
        return parent
    end
    mass, el = get_mass_name(parent)
    el_index = [i for i in 1:length(element_symbols)
                if element_symbols[i] == string(el)][1]
    if decay_type == "alpha" || decay_type == "A"
        return string(mass - 4) * element_symbols[el_index - 2]
    end
    if decay_type == "beta-minus" || decay_type == "B-"
        return string(mass) * element_symbols[el_index + 1]
    end
    if decay_type == "beta-plus" || decay_type == "B+"
        return string(mass) * element_symbols[el_index - 1]
    end
    if decay_type == "neutron" || decay_type == "N"
        return string(mass-1) * el
    end
    if decay_type == "proton" || decay_type == "P"
        return string(mass-1) * element_symbols[el_index - 1]
    end
    if decay_type == "double neutron" || decay_type == "2N"
        return string(mass-2) * el
    end
    if decay_type == "double proton" || decay_type == "2P"
        return string(mass-2) * element_symbols[el_index - 2]
    end
    if decay_type == "double neutron" || decay_type == "2N"
        return string(mass-2) * element_symbols[el_index - 1]
    end
    return decay_type * " not yet accounted for"
end

get_daughter_nuclide (generic function with 1 method)

In [80]:
livechart_abbreviation_decay_types = Dict([
    "A" => "alpha", "B-" => "beta-minus", "B+" => "beta-plus", "N" => "neutron", "P" => "proton"
])
unique(all_live_chart_data[!,"decay_1"])

21-element Vector{String}:
 "B-"
 " "
 "N"
 "2N"
 "P"
 "A"
 "EC"
 "EC+B+"
 "2P"
 "B+P"
 "B-N"
 "B-2N"
 "B+"
 "2B-"
 "ECP+EC2P"
 "2EC"
 "IT"
 "ECP"
 "2B+"
 "SF"
 "ECSF"

Each decay route is stored as its own string of the form isotope,decay type,probability,average energy released, daughter isotope. 

For example,

"3H,B-,100%,5.69keV,3He"

 means that 100% of tritium decays are beta-minus decay, and on average the produced beta particles will have a kinetic energy of 5.69 keV.

In [81]:
comprehensive_decay_chains_dir = parent_dir * "ExportedData\\Comprehensive_decay_chains\\"
decay_file_names = readdir(comprehensive_decay_chains_dir)
decay_chain = CSV.read(comprehensive_decay_chains_dir * decay_file_names[200], DataFrame)[!,1]

3-element Vector{String}:
 "113Mo,B-,100%,3430.0keV,0.08sec" ⋯ 220 bytes ⋯ "3571968e23seconds,113In,stable"
 "113Mo,B-N, %,B-N not yet accoun" ⋯ 25 bytes ⋯ " for ,B-N not yet accounted for"
 "113Mo,B-,100%,3430.0keV,0.08sec" ⋯ 63 bytes ⋯ " for ,B-N not yet accounted for"

In [82]:
CSV.read(comprehensive_decay_chains_dir * "238U.csv", DataFrame)[!, 1]

25-element Vector{String}:
 "238U,A,100%,4187.13keV,1.409963" ⋯ 551 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,SF,0.0000545%,SF not yet a" ⋯ 28 bytes ⋯ "d for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 160 bytes ⋯ " for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 161 bytes ⋯ " for ,Mg not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 218 bytes ⋯ "or ,24NE not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 210 bytes ⋯ " for ,SF not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 261 bytes ⋯ "for ,14C not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 549 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 546 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 540 bytes ⋯ "11955686.4seconds,206Pb,stable"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 419 bytes ⋯ "for ,B-A not yet accounted for"
 "238U,A,100%,4187.13keV,1.409963" ⋯ 538 bytes ⋯ "11955686.4seconds,206

In [83]:
function parse_decay_chain(decay_chain)
    arr = split(decay_chain, ",")
    decay_steps = Int(floor(length(arr) / 5))
    decays = Vector{Any}(undef, decay_steps + 1)
    for decay_step in 1 : decay_steps
       decays[decay_step] = arr[5 * (decay_step - 1) + 1 : 5*decay_step] 
    end
    decays[decay_steps + 1] = arr[5 * decay_steps + 1 : end]
    return decays
end
parse_decay_chain(decay_chain)

1-element Vector{Any}:
 [["113Mo,B-,100%,3430.0keV,0.08seconds,113Tc,B-,100%,2712.98keV,0.152seconds,113Ru,B-,100%,2531.81keV,0.8seconds,113Rh,B-,100%,1580.16keV,2.8seconds,113Pd,B-,100%,1362.75keV,93seconds,113Ag,B-,100%,762.85keV,19332seconds,113Cd,B-,100%,92.6keV,2.5371768483571968e23seconds,113In,stable", "113Mo,B-N, %,B-N not yet accounted for,B-N not accounted for ,B-N not yet accounted for", "113Mo,B-,100%,3430.0keV,0.08seconds,113Tc,B-N,2.1%,B-N not yet accounted for,B-N not accounted for ,B-N not yet accounted for"]]

In [84]:
decay_dict = Dict([])
for file in decay_file_names
    nuclide = split(file, ".csv")[1]
    file_path = comprehensive_decay_chains_dir * file
    decay_chains = CSV.read(file_path, DataFrame, stringtype = String)[!, 1]
    decay_chains = [parse_decay_chain(decay_chain) for decay_chain in decay_chains]
    decay_dict[nuclide] = decay_chains
end
decay_dict

Dict{Any, Any} with 3089 entries:
  "113Sn" => Vector{Any}[[SubString{String}["113Sn", "EC+B+", "100%", "EC+B+ no…
  "243Cf" => Vector{Any}[[SubString{String}["243Cf", "EC+B+", "86%", "EC+B+ not…
  "168Eu" => Vector{Any}[[SubString{String}["168Eu", "B-", "100%", "2168.0keV",…
  "160Dy" => Vector{Any}[[SubString{String}["160Dy", "stable"]]]
  "162Sm" => Vector{Any}[[SubString{String}["162Sm", "B-", "100%", "1383.0keV",…
  "57V"   => Vector{Any}[[SubString{String}["57V", "B-", "100%", "3558.89keV", …
  "208Po" => Vector{Any}[[SubString{String}["208Po", "A", "99.996%", "5114.7keV…
  "174Ir" => Vector{Any}[[SubString{String}["174Ir", "EC", "99.5%", "EC not yet…
  "144Gd" => Vector{Any}[[SubString{String}["144Gd", "EC+B+", "100%", "EC+B+ no…
  "80Sr"  => Vector{Any}[[SubString{String}["80Sr", "EC+B+", "100%", "EC+B+ not…
  "33Ar"  => Vector{Any}[[SubString{String}["33Ar", "EC+B+", "100%", "EC+B+ not…
  "220Pa" => Vector{Any}[[SubString{String}["220Pa", "A", "100%", "Energy not f…
  "108In" 

In [85]:
decay_dict["238U"][2]

2-element Vector{Any}:
 SubString{String}["238U", "SF", "0.0000545%", "SF not yet accounted for", "SF not accounted for "]
 SubString{String}["SF not yet accounted for"]

In [86]:
[chain for chain in decay_dict["238U"]
if ["SF not yet accounted for"] in chain]

3-element Vector{Vector{Any}}:
 [SubString{String}["238U", "SF", "0.0000545%", "SF not yet accounted for", "SF not accounted for "], SubString{String}["SF not yet accounted for"]]
 [SubString{String}["238U", "A", "100%", "4187.13keV", "1.4099634525447706e17seconds"], SubString{String}["234Th", "B-", "100%", "47.81keV", "2082240seconds"], SubString{String}["234Pa", "B-", "100%", "212.37keV", "24120seconds"], SubString{String}["234U", "SF", "0.00000000164%", "SF not yet accounted for", "SF not accounted for "], SubString{String}["SF not yet accounted for"]]
 [SubString{String}["238U", "A", "100%", "4187.13keV", "1.4099634525447706e17seconds"], SubString{String}["234Th", "B-", "100%", "47.81keV", "2082240seconds"], SubString{String}["234Pa", "B-", "100%", "212.37keV", "24120seconds"], SubString{String}["234U", "A", "100%", "4759.43keV", "7747225326762.336seconds"], SubString{String}["230Th", "SF", "0.000000000004%", "SF not yet accounted for", "SF not accounted for "], SubString{String}["

In [87]:
decay_dict["238U"][2]

2-element Vector{Any}:
 SubString{String}["238U", "SF", "0.0000545%", "SF not yet accounted for", "SF not accounted for "]
 SubString{String}["SF not yet accounted for"]

In [88]:
function all_decay_modes_accounted_for(decay_chain) 
    for decay_step in decay_chain
        for item in decay_step
            if length(item) > 21
                if occursin("not yet accounted for", item)
                    return false
                end
            end
        end
    end
    return true
end

function get_decay_info(decay_chain)
    if any([any([occursin(" ", element) || occursin("nothing", element) 
             for element in step]) for step in decay_chain])
        return "Insufficient data"
    end
    if all_decay_modes_accounted_for(decay_chain)
        decay_energies = [parse(Float64, decay_step[4][1:end-3]) 
                            for decay_step in decay_chain 
                                if length(decay_step) > 3]
        decay_probabilities = [parse(Float64, decay_step[3][1:end-1]) / 100 
                                for decay_step in decay_chain  
                                    if length(decay_step) > 2] 
        e_folding_times = [parse(Float64, decay_step[5][1:end-7]) / log(2)
                            for decay_step in decay_chain 
                                if length(decay_step) > 3] 
        return decay_probabilities, decay_energies, e_folding_times
    end
    return "Decay mode(s) not yet accounted for"
end
get_decay_info(decay_dict["210Pb"][3])

([1.0, 1.32e-6, 1.0], [6.08, 0.00617, 538.37], [1.0106998575252168e9, 624740.0438824259, 363.73227370892545])

Which is correct as seen by the cell below :) 

In [89]:
decay_dict["210Pb"][2]

4-element Vector{Any}:
 SubString{String}["210Pb", "A", "0.0000019%", "0.0keV", "700563756.6359425seconds"]
 SubString{String}["206Hg", "B-", "100%", "400.35keV", "499.2seconds"]
 SubString{String}["206Tl", "B-", "100%", "538.37keV", "252.12seconds"]
 SubString{String}["206Pb", "stable"]

However, this only works for some decay chains

In [90]:
get_decay_info(decay_dict["238U"][2])

"Insufficient data"

I derived the decay rate of the n-th generation nuclide in a decay chain here:

https://www.overleaf.com/project/627613def6b848465d85e5bb

https://github.com/MarcosP7635/Math-for-Energy/blob/main/main.tex

file:///C:/Users/engin/Downloads/Math_for_Energy(4).pdf 

In [91]:
factor(t, e_folding_time) = 1 - exp(t / -e_folding_time)

factor (generic function with 1 method)

In [92]:
a_prod(e_folding_times, t) = prod([factor(t, e_folding_times[k])
                                   for k in 2:length(e_folding_times)])

a_prod (generic function with 1 method)

In [93]:
r(e_fold_times, t) = (a_prod(e_fold_times, t) * exp(t / -e_fold_times[1])
                      / e_fold_times[1])


r (generic function with 1 method)

In [94]:
λ_0, λ_1, λ_2, t = @variables λ_0, λ_1, λ_2, t
expected_output = (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * exp(t / -λ_0) / λ_0

((1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [95]:
e_folding_times = [λ_0, λ_1, λ_2]
r(e_folding_times, t)
Test.@test r(e_folding_times, t) - expected_output == 0

Test Passed
  Expression: r(e_folding_times, t) - expected_output == 0
   Evaluated: 0 == 0

It works! :) 

The average beta decay energy is given in keV. 

In [96]:
joules_per_keV = Rational(uconvert(@u_str("J"), 1 * @u_str("keV")) / @u_str("J"))

337//2103388558093277156

Avogadro's number

In [97]:
N_A = 6.02214076e23
conversion_factor = Rational(joules_per_keV * N_A)

6475021031458043//67108864

Now we calculate the power in watts

In [98]:
power_per_mole(decay_rate, decay_energy) = decay_rate * decay_energy * conversion_factor

power_per_mole (generic function with 3 methods)

In [99]:
function power_per_mole(decay_probabilities, decay_energies, e_folding_times, t)
    l = length(decay_probabilities)
    if l == 1
        return decay_probabilities .* power_per_mole(r(e_folding_times, t), decay_energies[end])
    end
    output = decay_probabilities[end] * sum(power_per_mole(r(e_folding_times, t),
                                             decay_energies[end]))
    for i in 2:l
        index = l - i + 1
        output = decay_probabilities[index] * (output + 
                    power_per_mole(r(e_folding_times[1:index], t), decay_energies[index]))
    end
    return output
end

power_per_mole (generic function with 3 methods)

In [100]:
E_0, E_1, E_2, p_0, p_1, p_2, t = @variables E_0, E_1, E_2, p_0, p_1, p_2, t
expected_output = conversion_factor * (
    p_0 * (E_0 * exp(t / -λ_0) / λ_0 + 
        p_1 * ((1 - exp(t / -λ_1)) * E_1 * exp(t / -λ_0) / λ_0 + 
            p_2 * (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * E_2 * exp(t / -λ_0) / λ_0
        )
    )
)          

(6475021031458043//67108864)*p_0*(p_1*((E_1*(1 - exp(t / (-λ_1)))*exp(t / (-λ_0))) / λ_0 + (E_2*p_2*(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0) + (E_0*exp(t / (-λ_0))) / λ_0)

In [101]:
decay_energies = [E_0, E_1, E_2]
decay_probabilities = [p_0, p_1, p_2]
Test.@test Symbolics.simplify(power_per_mole(decay_probabilities,
                                decay_energies, e_folding_times, t) 
            - expected_output) == 0

Test Passed
  Expression: Symbolics.simplify(power_per_mole(decay_probabilities, decay_energies, e_folding_times, t) - expected_output) == 0
   Evaluated: 0 == 0

In [102]:
typeof([4.9288656281454315e9]) == Vector{Float64} 

true

The function named power_all_decay_modes calculates the power in watts produced by one mole of a given isotope after a given time in seconds. 
Note that so far, this only includes alpha decay, beta decay, neutron decay, and proton decay. 

In [123]:
function power_all_decay_modes(isotope, time)
    decay_chains_given_isotope = decay_dict[isotope]
    power = 0
    for decay_chain in decay_chains_given_isotope
        test = get_decay_info(decay_chain)
        if (test != "Decay mode(s) not yet accounted for" 
            && test[1] != Float64[] && test != "Insufficient data")
            decay_probabilities, decay_energies, e_folding_times = test
            sub_power = power_per_mole(decay_probabilities, decay_energies, 
                                        e_folding_times, time)
            if (typeof(sub_power) == Vector{Float64} || 
                typeof(sub_power) == Vector{BigFloat})
                power += sub_power[1]
            else
                power += sub_power
            end
        end
    end
    return power
end

power_all_decay_modes (generic function with 1 method)

In [107]:
zero_gen_isotopes = [split(file, ".csv")[1] for file in decay_file_names]
power_dict = Dict([])
for isotope in zero_gen_isotopes
    power_dict[isotope] = power_all_decay_modes(isotope, .1)
end
power_dict

Dict{Any, Any} with 3089 entries:
  "113Sn" => 0
  "243Cf" => 5.79142e7
  "168Eu" => 5.20787e11
  "160Dy" => 0
  "162Sm" => 3.75573e10
  "57V"   => 6.00011e11
  "208Po" => 0
  "174Ir" => 0
  "144Gd" => 0
  "80Sr"  => 0
  "33Ar"  => 0
  "220Pa" => 0
  "108In" => 0
  "252No" => 1.97183e11
  "254Es" => 2.69543e5
  "165Hf" => 0
  "67Mn"  => 1.62319e12
  "260Sg" => 228174.0
  "233Cm" => 0
  "103Rb" => 1.09346e12
  "32S"   => 0
  "80Zr"  => 0
  "228Th" => 11981.2
  "136Nd" => 0
  "51Ti"  => 1.67886e8
  ⋮       => ⋮

It takes 1 second to calculate the power of every decay chain for a given time value. 

In [117]:
exponents_for_time_array = -10:20/100:10
collect(exponents_for_time_array)

101-element Vector{Float64}:
 -10.0
  -9.8
  -9.6
  -9.4
  -9.2
  -9.0
  -8.8
  -8.6
  -8.4
  -8.2
  -8.0
  -7.8
  -7.6
   ⋮
   7.8
   8.0
   8.2
   8.4
   8.6
   8.8
   9.0
   9.2
   9.4
   9.6
   9.8
  10.0

In [119]:
time_array = [BigFloat(10)^x for x in exponents_for_time_array]

101-element Vector{BigFloat}:
 1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
 1.584893192461110892181946192724010648088344291738932269688212001583469968929013e-10
 2.511886431509582165915027997951851781486256187100727119702478521515105210922273e-10
 3.981071705534969251016450836283920198412114190191191509354272293588653173320875e-10
 6.309573444801942817342773037899355684778124488909748123578127485731705747260038e-10
 1.000000000000000000000000000000000000000000000000000000000000000000000000000005e-09
 1.584893192461110892181946192724010648088344291738932269688212001583469968929009e-09
 2.511886431509582165915027997951851781486256187100727119702478521515105210922281e-09
 3.981071705534969251016450836283920198412114190191191509354272293588653173320875e-09
 6.309573444801942817342773037899355684778124488909748123578127485731705747260006e-09
 9.999999999999999999999999999999999999999999999999999999999999999999999999999956e-09
 1.584893192461114133457

In [124]:
power_time_series = [[power_all_decay_modes(isotope, time)
                        for time in time_array] for isotope in zero_gen_isotopes]; 
power_time_series_dict = Dict([])
stop = length(power_time_series)
for i in 1:stop
        power_time_series_dict[zero_gen_isotopes[i]] = power_time_series[i]
end
power_time_series_dict

Dict{Any, Any} with 3089 entries:
  "113Sn" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "243Cf" => BigFloat[5.79205e+07, 5.79205e+07, 5.79205e+07, 5.79205e+07, 5.792…
  "168Eu" => BigFloat[7.24963e+11, 7.24963e+11, 7.24963e+11, 7.24963e+11, 7.249…
  "160Dy" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "162Sm" => BigFloat[3.85388e+10, 3.85388e+10, 3.85388e+10, 3.85388e+10, 3.853…
  "57V"   => BigFloat[7.43792e+11, 7.43792e+11, 7.43792e+11, 7.43792e+11, 7.437…
  "208Po" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "174Ir" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "144Gd" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "80Sr"  => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "33Ar"  => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "220Pa" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  "108In" => [0, 0, 0, 0, 

In [133]:
function plot_power_time_series(nuclide, time_array)
    power_time_series = Float64.(power_time_series_dict[nuclide])
    time_array = Float64.(time_array)
    plot(scatter(x = time_array, y = power_time_series, mode = "markers"), 
    Layout(xaxis_type = "log"))#, yaxis_type = "log"))
end
plot_power_time_series("", time_array)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, and xaxis"

In [20]:
values(power_time_series_dict)

ValueIterator for a Dict{Any, Any} with 3192 entries. Values:
  BigFloat[19.10255687989461236685044743340581239013755137158583044737971404894…
  BigFloat[587251.3417034893162085043426417597518662359998193764706657792645493…
  BigFloat[1.902694346497644245150516668616150204766859907155112602001096115937…
  BigFloat[1.409403220047922261847711779732440399370103109599795278937175528118…
  BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, …
  BigFloat[1.087253912424558971525400118494233169459749268892055916591742346843…
  BigFloat[4.160990031372746078034922899154623194906258702410810853183757204621…
  BigFloat[4.816947714556571279120306684220982236639560494868173793622995720059…
  BigFloat[1.418862302097725720321943105981181420127809995803572798046172609174…
  BigFloat[59664.29435929451363606820111435521224661824346134398145632086382050…
  BigFloat[2.199646643180580411020156654991285400502149815425932111723296924897…
  BigFloat[4.4817436926458402547613507994885068

In [135]:
power_matrix = Matrix{Union{Float64, BigFloat, Int64}}(undef, stop, length(time_array))
for row in 1:stop
    power_matrix[row, :] = power_time_series_dict[zero_gen_isotopes[row]]
end
power_df = DataFrame(power_matrix, string.(time_array))

,1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
,Union…
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [136]:
power_df[!, "Parent Isotope"] = zero_gen_isotopes

3089-element Vector{SubString{String}}:
 "100Ag"
 "100Cd"
 "100In"
 "100Kr"
 "100Mo"
 "100Nb"
 "100Pd"
 "100Rb"
 "100Rh"
 "100Ru"
 "100Sn"
 "100Sr"
 "100Tc"
 ⋮
 "99Pd"
 "99Rb"
 "99Rh"
 "99Ru"
 "99Sn"
 "99Sr"
 "99Tc"
 "99Y"
 "99Zr"
 "9Be"
 "9C"
 "9Li"

In [138]:
CSV.write(cross_section_dir * "All_decay_modes_power_time_series_101_steps.csv", power_df)

"C:\\Cross-Section-Data\\All_decay_modes_power_time_series_101_steps.csv"

https://drive.google.com/file/d/1aOk0XagAuHJ9FUSMbESXMMc5aSb0LOwH/view?usp=sharing

Now to find the highest power isotopes as a function of time. 

In [139]:
str_time_array = string.(time_array)
max_power_isotopes = [zero_gen_isotopes[argmax(power_df[!, t])]
                    for t in str_time_array]
max_power_values = [power_df[argmax(power_df[!, t]), t] for t in str_time_array]

101-element Vector{BigFloat}:
     5.295743860585541362268520139389228751761384653291691858677615325622021706240465e+18
     5.293984329295320343119224724409779775614548824082189372740317516423370074307745e+18
     5.291196857565264019991032499487239514170612512468014747118778822631142410218467e+18
     5.286782018716108112187576570796707580035701959008717281518854610024788706285431e+18
     5.27979251495722704986804568770258036683604920338031890118081266558958486433078e+18
     5.268733821550779975201045475168908266836550265314526319704499479115192429777388e+18
     5.251254400687192289428779472811884433664809105652002594641430319187435994278412e+18
     5.223670093373546061652249717261244072720616261322035732836882845731150721019072e+18
     5.18024841506855229365360434470943579151988778454391498698467691338802534941132e+18
     5.112167843462723562419444031384616306869701594307297977087044937938532801848197e+18
     5.00609547832502503485332283468772861857336625754181844766481651132

In [140]:
max_power_df = DataFrame(Time_seconds = time_array, Power_watts = max_power_values,
Isotope = max_power_isotopes, stringtype = string)

,Time_seconds
,BigFloat
1,1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
2,1.584893192461110892181946192724010648088344291738932269688212001583469968929013e-10
3,2.511886431509582165915027997951851781486256187100727119702478521515105210922273e-10
4,3.981071705534969251016450836283920198412114190191191509354272293588653173320875e-10
5,6.309573444801942817342773037899355684778124488909748123578127485731705747260038e-10
6,1.000000000000000000000000000000000000000000000000000000000000000000000000000005e-09
7,1.584893192461110892181946192724010648088344291738932269688212001583469968929009e-09
8,2.511886431509582165915027997951851781486256187100727119702478521515105210922281e-09
9,3.981071705534969251016450836283920198412114190191191509354272293588653173320875e-09


In [141]:
plot(scatter(max_power_df, x=:Time_seconds, y=:Power_watts, mode="markers",
text=:Isotope, color=:Isotope),
Layout(xaxis_type = "log", yaxis_type = "log", 
color = max_power_df[!, "Isotope"], show_scale =true))

data: [
  "scatter with fields color, mode, text, type, x, and y"
]

layout: "layout with fields color, margin, show, template, xaxis, and yaxis"

In [27]:
uconvert(@u_str("J"), 16 * @u_str("MeV") * N_A)

1.5437653139729602e12 J

In [28]:
decay_chain_energies = [sum(df[!, "Average beta-decay energy"])
                        for df in all_decay_chains]
maximum(decay_chain_energies) * conversion_factor

1.8561616629085276e12

In [29]:
best_decay_chain_index = argmax(decay_chain_energies)
zero_gen_isotopes[best_decay_chain_index]

"218Tl"

In [30]:
decay_chain_energy_df = DataFrame(Parent_Isotope = zero_gen_isotopes, 
Energy_keV = decay_chain_energies)

,Parent_Isotope,Energy_keV
,String,Float64
1,100Ag,22.76
2,100Cd,17.07
3,100In,11.38
4,100Kr,45.52
5,100Mo,11.38
6,100Nb,17.07
7,100Pd,25.15
8,100Rb,39.83
9,100Rh,349.15


In [31]:
CSV.write(parent_dir * "ExportedData\\Beta_decay_chain_energies.csv", decay_chain_energy_df)

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\ExportedData\\Beta_decay_chain_energies.csv"